In [1]:
import sys
sys.path.insert(0, '../python_scripts')
import json
import glob
import random
import matplotlib.pyplot as plt
from tqdm import tqdm_notebook as tq
import matplotlib.patches as patches
import numpy as np
from PIL import Image
import utils
import importlib
import threading
import meta
import heatmap
import os
import heatmap
from shutil import copyfile
import time
import h_gen,meta
from meta import *
from h_gen import *
from utils import *
import gc 
from scipy.misc import imread
import cv2
from sklearn import cluster
from skimage.measure import compare_ssim
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
from PIL import Image

Using Theano backend.


In [20]:
import os

TRAIN_PATH = "./train"
RELABELS_PATH = "relabels.csv"

os.mkdir("{}/{}".format(TRAIN_PATH, "revise"))

with open(RELABELS_PATH) as f:
    for line in f:
        cols = line.split()
        src = "{}/{}/{}.jpg".format(TRAIN_PATH, cols[1], cols[0])
        dst = "{}/{}/{}.jpg".format(TRAIN_PATH, cols[2], cols[0])

        try:
            os.rename(src, dst)

        except FileNotFoundError:
            print("{} not found".format(src))

./train/NoF/img_00028.jpg not found
./train/NoF/img_00076.jpg not found
./train/ALB/img_00248.jpg not found
./train/OTHER/img_00485.jpg not found
./train/ALB/img_00568.jpg not found
./train/NoF/img_00673.jpg not found
./train/YFT/img_00739.jpg not found
./train/YFT/img_00752.jpg not found
./train/NoF/img_00819.jpg not found
./train/YFT/img_00890.jpg not found
./train/NoF/img_00904.jpg not found
./train/YFT/img_01007.jpg not found
./train/YFT/img_01087.jpg not found
./train/NoF/img_01120.jpg not found
./train/ALB/img_01363.jpg not found
./train/YFT/img_01410.jpg not found
./train/OTHER/img_01452.jpg not found
./train/YFT/img_01532.jpg not found
./train/YFT/img_01610.jpg not found
./train/OTHER/img_01678.jpg not found
./train/NoF/img_01989.jpg not found
./train/NoF/img_02044.jpg not found
./train/ALB/img_02086.jpg not found
./train/OTHER/img_02099.jpg not found
./train/YFT/img_02199.jpg not found
./train/NoF/img_02232.jpg not found
./train/SHARK/img_02271.jpg not found
./train/NoF/img_02

In [21]:
metadata = meta.create_metadata()

['../JSONS/alb_labels.json', '../JSONS/bet_labels.json', '../JSONS/dol_labels.json', '../JSONS/lag_labels.json', '../JSONS/other_labels.json', '../JSONS/shark_labels.json', '../JSONS/yft_labels.json']


In [5]:
metadata['img_02388.jpg']

{'class': 'ALB',
 'cluster': 1,
 'code': 2,
 'filename': 'img_02388.jpg',
 'height': 720,
 'path': './train/ALB/img_02388.jpg',
 'rectangles': [{'height': 55.809407809078834,
   'width': 143.67953925316039,
   'x': 463.09934139448393,
   'y': 480.91085452504103}],
 'width': 1280}

### Let's first do a classification with the images sizes.

In [6]:
for key, v in metadata.items():
    folder = "./train/" + v["class"] + "/"
    destination = "{}{}_{}/".format(folder, v["height"], v["width"])
    mk(destination)
    os.rename(v["path"], destination + v["filename"])

In [7]:
for folder in get_all_images(regex = "./train/*/*/"):
    break
print(folder)   

./train/ALB/670_1192/


In [32]:
def compare(img, img2):
    #return compare_ssim(img,img2)
    return np.mean(np.abs(img - img2))

In [36]:
# We loop over the folder containing the images.
def compare(img, img2):
    #return compare_ssim(img,img2)
    return np.mean(np.abs(img - img2))
for folder in tq(get_all_images(regex = "./train/*/*/")):
    print(folder)
    train_files = get_all_images(regex=folder + "*.jpg")
    train = np.array([img_to_array(Image.open(img).resize((224,224))) for img in train_files])

    # Resize the images to speed it up.

    custom_train = []
    for img in train:
        im = (img - img.mean()) / img.std()
        custom_train.append(im.reshape((224*224*3,)))

    train = np.array(custom_train)

    l = len(train)
    
    if len(train) <=4:
        continue
    
    distances = np.zeros((l,l))
    
    # We compute the distance matrix
    for i in tq(range(len(train))):
        for j in range(len(train)):
            distances[i,j] = compare(train[i], train[j])
    
    # We compute the clusters
    cls = cluster.DBSCAN(metric='precomputed', min_samples=5, eps=0.6)
    y = cls.fit_predict(distances)
    
    # We move the images in the good folder.
    for path, cluster_idx in zip(train_files, y.tolist()):
        dest_folder = folder + str(cluster_idx) + "/"
        mk(dest_folder)
        dest_path = dest_folder + path.split("/")[-1]
        os.rename(path, dest_path)

./train/ALB/670_1192/
./train/ALB/700_1244/
./train/ALB/718_1276/
./train/ALB/720_1280/
./train/ALB/750_1280/
./train/ALB/750_1334/
./train/ALB/854_1518/
./train/ALB/924_1280/
./train/ALB/974_1280/
./train/BET/670_1192/
./train/BET/700_1244/
./train/BET/718_1276/
./train/BET/720_1280/
./train/BET/750_1280/
./train/BET/854_1518/
./train/BET/974_1280/
./train/DOL/720_1280/
./train/DOL/750_1280/
./train/DOL/974_1280/
./train/LAG/720_1280/
./train/NoF/670_1192/
./train/NoF/700_1244/
./train/NoF/718_1276/
./train/NoF/720_1280/
./train/NoF/750_1280/
./train/NoF/750_1334/
./train/NoF/854_1518/
./train/NoF/974_1732/
./train/OTHER/720_1280/
./train/OTHER/750_1280/
./train/OTHER/974_1280/
./train/SHARK/720_1280/
./train/SHARK/750_1280/
./train/SHARK/854_1518/
./train/SHARK/974_1280/
./train/YFT/670_1192/
./train/YFT/700_1244/
./train/YFT/718_1276/
./train/YFT/720_1280/
./train/YFT/750_1280/
./train/YFT/854_1518/
./train/YFT/924_1280/
./train/YFT/974_1280/



In [35]:
distances

array([[ 0.        ,  0.11646245,  0.72772646, ...,  0.75299323,
         0.75305879,  0.7463221 ],
       [ 0.11646245,  0.        ,  0.73529834, ...,  0.7597602 ,
         0.75947171,  0.75237507],
       [ 0.72772646,  0.73529834,  0.        , ...,  0.28193817,
         0.20976514,  0.24870646],
       ..., 
       [ 0.75299323,  0.7597602 ,  0.28193817, ...,  0.        ,
         0.19873011,  0.15850423],
       [ 0.75305879,  0.75947171,  0.20976514, ...,  0.19873011,
         0.        ,  0.12003927],
       [ 0.7463221 ,  0.75237507,  0.24870646, ...,  0.15850423,
         0.12003927,  0.        ]])

In [310]:
def get_fishes(GOOD_TO_KEEP):
    img_names = []
    for element in GOOD_TO_KEEP:
        for ele in element:
            #print(ele)
            for e in ele['subdir']:
                for img_name in e['img_name']:
                    #print(img_name)
                    img_names.append(img_name)
    return img_names
def concat_sub_list(objs):
    concat = []
    for obj in objs:
        for img in obj:
            concat.append(img)
    return set(concat)

### YTF SELECTION

In [311]:
def filter_YFT(dir,obj,img_names):
    if dir ==  '670_1192':
        if obj == '0':
            return ['img_05183.jpg']
    elif dir == '700_1244':
        if obj == '0':
            return ['img_06323.jpg']        
        elif obj == '1':
            return ['img_05302.jpg']
        else:
            print('ERROR : '+obj)
    elif dir == '718_1276':
        if obj == '0':
            return ['img_00047.jpg']        
        elif obj == '1':
            return ['img_00572.jpg']   
        if obj == '2':
            return ['img_01959.jpg']
        else:
            print('ERROR : '+obj)        
    elif dir == '720_1280':
        if obj == '0':
            return ['img_00021.jpg','img_02863.jpg']        
        elif obj == '1':
            return ['img_00286.jpg','img_01113.jpg','img_04825.jpg']   
        elif obj == '2':
            return ['img_01469.jpg']
        elif obj == '3':
            return ['img_02014.jpg']        
        elif obj == '4':
            return ['img_01358.jpg']   
        elif obj == '5':
            return ['img_06156.jpg'] 
        elif obj == '-1':
            return img_names
        else:
            print('ERROR : '+obj)        
    elif dir == '750_1280':
        if obj == '0':
            return ['img_07055.jpg']        
        elif obj == '1':
            return ['img_01472.jpg']   
        elif obj == '2':
            return ['img_04591.jpg','img_01323.jpg']
        elif obj == '3':
            return ['img_01375.jpg','img_03485.jpg']        
        elif obj == '4':
            return ['img_07369.jpg']   
        elif obj == '5':
            return ['img_02255.jpg'] 
        elif obj == '6':
            return ['img_06502.jpg','img_01749.jpg']        
        elif obj == '7':
            return ['img_02958.jpg']   
        elif obj == '8':
            return ['img_07609.jpg','img_06487.jpg']
        elif obj == '9':
            return ['img_05380.jpg']        
        elif obj == '10':
            return ['img_02962.jpg']   
        elif obj == '-1':
            return ['img_07290.jpg','img_00137.jpg','img_04364.jpg']
        else:
            print('ERROR : '+obj)        
    elif dir == '854_1518':
        if obj == '0':
            return ['img_05122.jpg'] 
    elif dir == '924_1280':
        if obj == '0':
            return ['img_03607.jpg','img_05506.jpg']   
        elif obj == '1':
            return ['img_04367.jpg'] #I think fishes in this cluster doesnt have good labels
        else:
            print('ERROR : '+obj)        
    elif dir == '974_1280':
        if obj == '0':
            return ['img_00362.jpg','img_01076.jpg']        
        elif obj == '1':
            return ['img_00573.jpg']   
        elif obj == '2':
            return ['img_03787.jpg','img_01623.jpg']
        elif obj == '3':
            return ['img_04756.jpg']        
        elif obj == '-1':
            return ['img_02642.jpg','img_06152.jpg']
        else:
            print('ERROR : '+obj)      
    else:
        print(dir)
        print('ERROR')
    

In [312]:
YFT_path = './train/YFT/'
GOOD_YFT = [[{'path':YFT_path+dir,'subdir':[{'img_name':filter_YFT(dir,obj,os.listdir(YFT_path+dir+'/'+obj)),'cluster':obj} for obj in os.listdir(YFT_path+dir)]}] for dir in os.listdir(YFT_path)]

In [313]:
YFT_json = {'YFT':get_fishes(GOOD_YFT)}

In [314]:
YFT_json

{'YFT': ['img_05183.jpg',
  'img_06323.jpg',
  'img_05302.jpg',
  'img_00047.jpg',
  'img_00572.jpg',
  'img_01959.jpg',
  'img_00114.jpg',
  'img_00522.jpg',
  'img_00778.jpg',
  'img_00969.jpg',
  'img_01254.jpg',
  'img_02075.jpg',
  'img_03081.jpg',
  'img_03560.jpg',
  'img_03921.jpg',
  'img_04533.jpg',
  'img_05171.jpg',
  'img_05321.jpg',
  'img_00021.jpg',
  'img_02863.jpg',
  'img_00286.jpg',
  'img_01113.jpg',
  'img_04825.jpg',
  'img_01469.jpg',
  'img_02014.jpg',
  'img_01358.jpg',
  'img_06156.jpg',
  'img_07290.jpg',
  'img_00137.jpg',
  'img_04364.jpg',
  'img_07055.jpg',
  'img_01472.jpg',
  'img_02962.jpg',
  'img_04591.jpg',
  'img_01323.jpg',
  'img_01375.jpg',
  'img_03485.jpg',
  'img_07369.jpg',
  'img_02255.jpg',
  'img_06502.jpg',
  'img_01749.jpg',
  'img_02958.jpg',
  'img_07609.jpg',
  'img_06487.jpg',
  'img_05380.jpg',
  'img_05122.jpg',
  'img_03607.jpg',
  'img_05506.jpg',
  'img_04367.jpg',
  'img_02642.jpg',
  'img_06152.jpg',
  'img_00362.jpg',
  'im

### SQUARK SELECTION

In [315]:
def filter_SHARK(dir,obj,img_names):
    if dir == '720_1280':
        if obj == '0':
            return ['img_00033.jpg','img_02867.jpg','img_02944.jpg','img_06467.jpg']        
        elif obj == '-1':
            return img_names
        else:
            print('ERROR : '+obj)        
    elif dir == '750_1280':
        if obj == '0':
            return ['img_03148.jpg','img_07498.jpg','img_06163.jpg','img_04682.jpg']        
        else:
            print('ERROR : '+obj)        
    elif dir == '854_1518':
        if obj == '0':
            return ['img_05122.jpg'] 
    elif dir == '924_1280':
        if obj == '0':
            return img_names
        else:
            print('ERROR : '+obj)        
    elif dir == '974_1280':
        if obj == '0':
            return ['img_01524.jpg','img_00245.jpg','img_03770.jpg','img_04837.jpg']        
        elif obj == '1':
            return ['img_04855.jpg','img_02114.jpg']   
        elif obj == '-1':
            return img_names
        else:
            print('ERROR : '+obj)      
    else:
        print(dir)
        print('ERROR')


In [316]:
SHARK_path = './train/SHARK/'
GOOD_SHARK = [[{'path':SHARK_path+dir,'subdir':[{'img_name':filter_SHARK(dir,obj,os.listdir(SHARK_path+dir+'/'+obj)),'cluster':obj} for obj in os.listdir(SHARK_path+dir)]}] for dir in os.listdir(SHARK_path)]

In [317]:
SHARK_json = {'SHARK':get_fishes(GOOD_SHARK)}

In [318]:
SHARK_json

{'SHARK': ['img_06082.jpg',
  'img_00033.jpg',
  'img_02867.jpg',
  'img_02944.jpg',
  'img_06467.jpg',
  'img_03148.jpg',
  'img_07498.jpg',
  'img_06163.jpg',
  'img_04682.jpg',
  'img_05122.jpg',
  'img_06006.jpg',
  'img_01524.jpg',
  'img_00245.jpg',
  'img_03770.jpg',
  'img_04837.jpg',
  'img_04855.jpg',
  'img_02114.jpg']}

In [319]:
### OTHER SELECTION

In [320]:
def filter_OTHER(dir,obj,img_names):
    if dir == '720_1280':
        if obj == '0':
            return ['img_00063.jpg','img_00359.jpg','img_01871.jpg','img_04371.jpg']
        elif obj == '1':
            return ['img_02051.jpg','img_04685.jpg','img_07751.jpg']# 51 dont know   
        elif obj == '2':
            return ['img_00226.jpg','img_01522.jpg']   
        elif obj == '3':
            return ['img_00631.jpg','img_05239.jpg']        
        elif obj == '4':
            return ['img_02031.jpg']   
        elif obj == '5':
            return ['img_02595.jpg']        
        elif obj == '6':
            return ['img_07227.jpg']        
        elif obj == '-1':
            return ['img_05607.jpg']# Maybe a YFT in back
        else:
            print('ERROR : '+obj)        
    elif dir == '750_1280':
        if obj == '0':
            return ['img_03148.jpg']        
        else:
            print('ERROR : '+obj)        
    elif dir == '854_1518':
        if obj == '0':
            return ['img_06878.jpg','img_06070.jpg'] 
        else:
            print('ERROR : '+obj)   
    elif dir == '974_1280':
        if obj == '0':
            return ['img_02773.jpg','img_02680.jpg']        
        elif obj == '-1':
            return ['img_01363.jpg','img_07839.jpg']
        else:
            print('ERROR : '+obj)      
    else:
        print(dir)
        print('ERROR')


In [321]:
OTHER_path = './train/OTHER/'
GOOD_OTHER = [[{'path':OTHER_path+dir,'subdir':[{'img_name':filter_OTHER(dir,obj,os.listdir(OTHER_path+dir+'/'+obj)),'cluster':obj} for obj in os.listdir(OTHER_path+dir)]}] for dir in os.listdir(OTHER_path)]

In [322]:
OTHER_json = {'OTHER':get_fishes(GOOD_OTHER)}

In [323]:
OTHER_json

{'OTHER': ['img_05607.jpg',
  'img_00063.jpg',
  'img_00359.jpg',
  'img_01871.jpg',
  'img_04371.jpg',
  'img_02051.jpg',
  'img_04685.jpg',
  'img_07751.jpg',
  'img_00226.jpg',
  'img_01522.jpg',
  'img_00631.jpg',
  'img_05239.jpg',
  'img_02031.jpg',
  'img_02595.jpg',
  'img_07227.jpg',
  'img_03148.jpg',
  'img_01363.jpg',
  'img_07839.jpg',
  'img_02773.jpg',
  'img_02680.jpg']}

### LAG SELECTION

In [324]:
def filter_LAG(dir,obj,img_names):
    if dir == '720_1280':
        if obj == '0':
            return ['img_00091.jpg','img_00359.jpg','img_01512.jpg','img_07142.jpg','img_00784.jpg','img_04678.jpg']
        elif obj == '1':
            return ['img_00176.jpg','img_04685.jpg','img_03940.jpg','img_03774.jpg']# 51 dont know   
        elif obj == '2':
            return ['img_05386.jpg','img_04953.jpg']   
        elif obj == '3':
            return ['img_06713.jpg','img_03771.jpg','img_02608.jpg']        
        elif obj == '-1':
            return concat_sub_list([img_names[:2],[img_names[-2]]])# Maybe a YFT in back
        else:
            print('ERROR : '+obj)        
    else:
        print(dir)
        print('ERROR')


In [325]:
LAG_path = './train/LAG/'
GOOD_LAG = [[{'path':LAG_path+dir,'subdir':[{'img_name':filter_LAG(dir,obj,os.listdir(LAG_path+dir+'/'+obj)),'cluster':obj} for obj in os.listdir(LAG_path+dir)]}] for dir in os.listdir(LAG_path)]

In [326]:
LAG_json = {'LAG':get_fishes(GOOD_LAG)}

In [327]:
LAG_json

{'LAG': ['img_02089.jpg',
  'img_01452.jpg',
  'img_05945.jpg',
  'img_00091.jpg',
  'img_00359.jpg',
  'img_01512.jpg',
  'img_07142.jpg',
  'img_00784.jpg',
  'img_04678.jpg',
  'img_00176.jpg',
  'img_04685.jpg',
  'img_03940.jpg',
  'img_03774.jpg',
  'img_05386.jpg',
  'img_04953.jpg',
  'img_06713.jpg',
  'img_03771.jpg',
  'img_02608.jpg']}

### DOL SELECTION

In [328]:
def filter_DOL(dir,obj,img_names):
    if dir == '720_1280':
        if obj == '0':
            return ['img_05444.jpg','img_01311.jpg','img_06204.jpg']
        elif obj == '1':
            return ['img_00325.jpg','img_05923.jpg','img_07751.jpg','img_07557.jpg','img_02752.jpg'] 
        else:
            print('ERROR : '+obj)        
    elif dir == '750_1280':
        if obj == '0':
            return ['img_05521.jpg','img_01850.jpg','img_06284.jpg','img_05165.jpg','img_03259.jpg']        
        else:
            print('ERROR : '+obj)        
    elif dir == '974_1280':
        if obj == '0':
            return ['img_02432.jpg','img_03843.jpg','img_05420.jpg','img_01055.jpg','img_02333.jpg','img_02529.jpg']  
        elif obj == '1':
            return ['img_02654.jpg','img_05923.jpg','img_06299.jpg','img_02798.jpg']
        elif obj == '-1':
            return img_names[1:]
        else:
            print('ERROR : '+obj)      
    else:
        print(dir)
        print('ERROR')

In [329]:
DOL_path = './train/DOL/'
GOOD_DOL = [[{'path':DOL_path+dir,'subdir':[{'img_name':filter_DOL(dir,obj,os.listdir(DOL_path+dir+'/'+obj)),'cluster':obj} for obj in os.listdir(DOL_path+dir)]}] for dir in os.listdir(DOL_path)]

In [330]:
DOL_json = {'DOL':get_fishes(GOOD_DOL)}

In [331]:
DOL_json

{'DOL': ['img_05444.jpg',
  'img_01311.jpg',
  'img_06204.jpg',
  'img_00325.jpg',
  'img_05923.jpg',
  'img_07751.jpg',
  'img_07557.jpg',
  'img_02752.jpg',
  'img_05521.jpg',
  'img_01850.jpg',
  'img_06284.jpg',
  'img_05165.jpg',
  'img_03259.jpg',
  'img_05446.jpg',
  'img_07265.jpg',
  'img_02432.jpg',
  'img_03843.jpg',
  'img_05420.jpg',
  'img_01055.jpg',
  'img_02333.jpg',
  'img_02529.jpg',
  'img_02654.jpg',
  'img_05923.jpg',
  'img_06299.jpg',
  'img_02798.jpg']}

### NoF SELECTION

In [332]:
def filter_NoF(dir,obj,img_names):
    if dir == '670_1192':
        if obj == '0':
            return ['img_00481.jpg','img_01311.jpg']
        elif obj == '-1':
            return img_names
        else:
            print('ERROR : '+obj)  
    elif dir == '700_1244':
        if obj == '0':
            return ['img_03009.jpg','img_01925.jpg']
        elif obj == '-1':
            return img_names  
        else:
            print('ERROR : '+obj)
    elif dir == '718_1276':
        if obj == '0':
            return ['img_00062.jpg','img_01588.jpg']
        elif obj == '-1':
            return img_names[1:-1] 
        else:
            print('ERROR : '+obj)  
    elif dir == '720_1280':
        if obj == '0':
            return ['img_00910.jpg','img_00022.jpg']
        elif obj == '1':
            return ['img_00135.jpg','img_00089.jpg','img_02439.jpg'] 
        if obj == '2':
            return ['img_00131.jpg','img_02381.jpg','img_00720.jpg']
        elif obj == '3':
            return ['img_00367.jpg','img_00387.jpg','img_02907.jpg'] 
        if obj == '4':
            return ['img_06816.jpg','img_02786.jpg']
        elif obj == '-1':
            return np.random.choice(img_names,size =10)
        else:
            print('ERROR : '+obj)        
    elif dir == '750_1280':
        if obj == '0':
            return ['img_00303.jpg']  
        elif obj == '1':
            return ['img_01710.jpg','img_04866.jpg'] 
        elif obj == '-1':
            return img_names        
        else:
            print('ERROR : '+obj)
    elif dir == '750_1334':
        if obj == '0':
            return ['img_03849.jpg','img_00266.jpg']  
        elif obj == '1':
            return ['img_06572.jpg','img_06322.jpg','img_07631.jpg'] 
        elif obj == '-1':
            return np.random.choice(img_names,size =5)      
        else:
            print('ERROR : '+obj)   
    elif dir == '854_1518':
        if obj == '0':
            return ['img_03908.jpg','img_03194.jpg'] 
        elif obj == '-1':
            return img_names    
        else:
            print('ERROR : '+obj)   
    elif dir == '974_1732':
        if obj == '0':
            return ['img_00094.jpg','img_03640.jpg','img_02805.jpg']  
        else:
            print('ERROR : '+obj)      
    else:
        print(dir,obj)

In [333]:
NoF_path = './train/NoF/'
GOOD_NoF = [[{'path':NoF_path+dir,'subdir':[{'img_name':filter_NoF(dir,obj,os.listdir(NoF_path+dir+'/'+obj)),'cluster':obj} for obj in os.listdir(NoF_path+dir)]}] for dir in os.listdir(NoF_path)]

In [334]:
NoF_json = {'NoF':get_fishes(GOOD_NoF)}

In [335]:
NoF_json

{'NoF': ['img_00339.jpg',
  'img_00826.jpg',
  'img_02441.jpg',
  'img_06402.jpg',
  'img_07512.jpg',
  'img_07902.jpg',
  'img_00481.jpg',
  'img_01311.jpg',
  'img_04063.jpg',
  'img_04501.jpg',
  'img_06329.jpg',
  'img_06375.jpg',
  'img_06499.jpg',
  'img_07115.jpg',
  'img_07274.jpg',
  'img_07544.jpg',
  'img_03009.jpg',
  'img_01925.jpg',
  'img_03536.jpg',
  'img_04886.jpg',
  'img_05998.jpg',
  'img_06083.jpg',
  'img_06601.jpg',
  'img_00062.jpg',
  'img_01588.jpg',
  'img_01497.jpg',
  'img_00105.jpg',
  'img_04178.jpg',
  'img_01497.jpg',
  'img_03025.jpg',
  'img_03765.jpg',
  'img_01155.jpg',
  'img_05012.jpg',
  'img_02870.jpg',
  'img_02970.jpg',
  'img_00910.jpg',
  'img_00022.jpg',
  'img_00135.jpg',
  'img_00089.jpg',
  'img_02439.jpg',
  'img_00131.jpg',
  'img_02381.jpg',
  'img_00720.jpg',
  'img_00367.jpg',
  'img_00387.jpg',
  'img_02907.jpg',
  'img_06816.jpg',
  'img_02786.jpg',
  'img_00568.jpg',
  'img_03002.jpg',
  'img_03257.jpg',
  'img_07464.jpg',
  'im

In [336]:
objects = [NoF_json, YFT_json, OTHER_json, DOL_json, LAG_json, SHARK_json]
pickle.dump(objects, open( "relabel_dataset.p", "wb" ) )

In [164]:
import os
from PIL import Image

TEST_PATH = "./test/"

dict_sizes = [{'size':Image.open(TEST_PATH+f).size,'img_name':f} for f in tq(os.listdir(TEST_PATH))]

In [165]:
for v in dict_sizes:
    folder = "./test/" 
    destination = "{}{}_{}/".format(folder, v['size'][0], v['size'][1])
    mk(destination)
    os.rename(folder+v['img_name'], destination + v['img_name'])

In [30]:
import warnings
warnings.filterwarnings('ignore')
def get_all_images(directory="./test/", regex = None):
    if regex is None:
        regex = directory+'/*/*.jpg'
    a = list(glob.glob(regex))
    return a
from skimage.measure import compare_ssim
def compare(img, img2):
    #return compare_ssim(img,img2)
    return np.mean(np.abs(img - img2))
train_files = []
for img_path in tq(get_all_images()):
    train_files.append(img_path.replace('\\','/'))
print(len(train_files))
train = np.array([img_to_array(Image.open(img).resize((224,224))) for img in tq(train_files)])
print(train.shape)
# Resize the images to speed it up.

custom_train = []
for img in train:
    im = (img - img.mean()) / img.std()
    custom_train.append(im.reshape((224*224*3,)))

train = np.array(custom_train)

l = len(train)

distances = np.zeros((l,l))

# We compute the distance matrix
for i in tq(range(len(train))):
    for j in range(len(train)):
        distances[i,j] = compare(train[i], train[j])



1000

(1000, 3, 224, 224)



In [166]:
# We compute the clusters
cls = cluster.DBSCAN(metric='precomputed', min_samples=2, eps=0.29)
y = cls.fit_predict(distances)

In [167]:
# We move the images in the good folder.
for path, cluster_idx in zip(train_files, y.tolist()):
    dest_folder = '/'.join(path.split('/')[:3])+'/'+ str(cluster_idx) + "/"
    mk(dest_folder)
    dest_path = dest_folder + path.split("/")[-1]
    os.rename(path, dest_path)

In [168]:
np.array([np.array([len(os.listdir(TEST_PATH+size+'/'+c)) for c in os.listdir(TEST_PATH+size) if c!='-1']).sum() for size in os.listdir(TEST_PATH)]).sum()


622.0

In [169]:
np.array([np.array([len(os.listdir(TEST_PATH+size+'/'+c)) for c in os.listdir(TEST_PATH+size) if c=='-1']).sum() for size in os.listdir(TEST_PATH)]).sum()

378

In [170]:
def find_cluster_for_img_name(img_name,input_cluster):
    all_imgs = glob.glob(input_cluster+'*/*/*')
    for img in all_imgs:
        img = img.replace('\\','/')
        if img_name in img:
            return img.split('/')[-2],img,'/'.join(img.split('/')[:-1])+'/'

In [171]:
def create_matrix_cluster(input_path,input_cluster):
    import collections
    import glob
    import os
    from PIL import Image
    data = collections.defaultdict(int)
    img_names = os.listdir(input_path)
    for img_name in tq(img_names):
        d = {}
        cluster,path,path_cluster = find_cluster_for_img_name(img_name,input_cluster)
        d['cluster']=cluster
        d['path']=path
        d['path_cluster']=path_cluster
        d['other_images']=[f for f in os.listdir(path_cluster) if f!=img_name]
        img = img_to_array(Image.open(path).resize((224,224)))
        img = (img - img.mean()) / img.std()
        dist = []
        for o_img in d['other_images']:
            img_o = img_to_array(Image.open(path_cluster+o_img).resize((224,224)))
            img_o = (img_o - img_o.mean()) / img_o.std()
            dist.append(compare(img, img_o))
        d['dist']=dist
        data[img_name]=d
    return data

In [172]:
data = create_matrix_cluster('../test/test','./test/')
pickle.dump(data, open( "test_clusterisation_2_0.3.p", "wb" ) )

In [173]:
keys = np.random.choice([key for key in data.keys()],2)
for key in keys:
    print(key,data[key])

img_01388.jpg {'path': './test/1280_720/66/img_01388.jpg', 'cluster': '66', 'path_cluster': './test/1280_720/66/', 'other_images': ['img_03181.jpg', 'img_03504.jpg', 'img_04118.jpg', 'img_06311.jpg'], 'dist': [0.1201657, 0.1292686, 0.21968397, 0.18822038]}
img_06658.jpg {'path': './test/1280_720/97/img_06658.jpg', 'cluster': '97', 'path_cluster': './test/1280_720/97/', 'other_images': ['img_03238.jpg', 'img_04127.jpg', 'img_05137.jpg', 'img_05471.jpg', 'img_05735.jpg', 'img_05817.jpg', 'img_05976.jpg', 'img_07893.jpg'], 'dist': [0.3255358, 0.11915534, 0.28974697, 0.1896123, 0.29752716, 0.18905799, 0.19089568, 0.18749869]}
